In [1]:
%load_ext autoreload
%autoreload 2
import sys
import platform
from pathlib import Path
from os import environ
if platform.system() == "Darwin": # Nat laptop
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/NeuroPy'])
    sys.path.extend(['/Users/nkinsky/Documents/UM/GitHub/Projects_sandbox'])
else:
    if environ["HOSTNAME"] == "lnx00004": # Nat Linux computer
        sys.path.extend(['/data/GitHub/NeuroPy'])
        sys.path.extend(['/data/GitHub/Projects_sandbox'])
    
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import scipy.signal as sg
import matplotlib.pyplot as plt
from neuropy.utils.mathutil import min_max_scaler
from scipy.ndimage import gaussian_filter1d, gaussian_filter
from copy import deepcopy
from tqdm import tqdm

from neuropy import plotting
from neuropy.analyses.placefields import Pf1D, Pf1Dsplit
from neuropy.analyses.oscillations import detect_theta_epochs
from neuropy.core.position import Position
from neuropy.core.epoch import Epoch
from neuropy.plotting.ratemaps import plot_ratemap
from neuropy.plotting.figure import Fig
from neuropy.utils.misc import flatten
from neuropy.io import BinarysignalIO

from Psilocybin.subjects import get_psi_dir

# These are crucial parameters - otherwise all text gets output with each letter in a word or heading as a separate unit
# instead of a text box
plt.rcParams['pdf.fonttype'] = 42
plt.rcParams['ps.fonttype'] = 42

# Specify Arial as font type - also crucial
plt.rcParams['font.sans-serif'] = 'Arial'
plt.rcParams['font.family'] = 'sans-serif'


/home/nkinsky/miniforge3/envs/NeuroPy/lib/python3.11/site-packages/outdated/utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.3, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [2]:
# Specify desired place to plot here.
plot_dir = Path("/Users/nkinsky/University of Michigan Dropbox/Nathaniel Kinsky/Manuscripts/Psilocybin/plots")

## Specify plotting parameters

In [3]:
animals = ["Finn", "Rey", "Rose", "Finn2"]
rasterize_scatter = True # This is crucial, default (False) will produce unweildy plots

In [14]:
ripple_thresh = 4 # 2.5 or 4

# Cut down Finn2 saline to 1hr?
chop_finn2_saline = False  # True = only use 1st hour of Finn2 saline, False = use all
finn2_append = "_1hrsalineonly" if chop_finn2_saline else ""

# ... OR only use 1hr Psilocybin for all
limit_to_1st_hr = False
chop_all_append = "_allsessions1hr" if limit_to_1st_hr else ""
finn2_append = "" if chop_all_append else finn2_append

## Load in data

In [15]:
df_grp = []
for ida, animal in enumerate(animals):
    file_use = get_psi_dir(animal, "Saline1").parent / "aggdata" / f"{animal.lower()}_rpl_features_thresh{'_'.join(str(ripple_thresh).split('.'))}{chop_all_append}.csv"
    df_animal = pd.read_csv(file_use, index_col=0)
    df_animal["Animal_name"] = animal
    df_animal["Animal"] = ida + 1
    df_grp.append(df_animal)

df_grp = pd.concat(df_grp, axis=0, ignore_index=True)
df_grp
    

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,Animal_name,Animal
0,0.9184,0.9992,0.9712,28540.330,197.917961,0.0808,168.181818,NaN,13.364659,Saline 1,Finn,1
1,69.9744,70.0368,70.0056,43174.117,319.979066,0.0624,221.212121,NaN,6.180823,Saline 1,Finn,1
2,70.4328,70.5104,70.4736,25572.940,223.388474,0.0776,166.666667,NaN,3.513697,Saline 1,Finn,1
3,89.4656,89.5320,89.5032,35710.438,278.840761,0.0664,171.212121,NaN,6.030679,Saline 1,Finn,1
4,120.9960,121.0720,121.0464,28893.555,223.479148,0.0760,187.878788,NaN,6.569296,Saline 1,Finn,1
...,...,...,...,...,...,...,...,...,...,...,...,...
14151,11503.6800,11503.7568,11503.7296,25091.900,182.681418,0.0768,162.121212,NaN,8.826762,Saline 2,Finn2,4
14152,11506.7480,11506.9304,11506.8040,29277.814,181.751974,0.1824,125.757576,NaN,8.691295,Saline 2,Finn2,4
14153,11509.3720,11509.4984,11509.4296,38501.550,302.989666,0.1264,186.363636,NaN,9.745170,Saline 2,Finn2,4
14154,11520.1152,11520.2144,11520.1496,49521.440,321.642974,0.0992,163.636364,NaN,20.808641,Saline 2,Finn2,4


In [16]:
# Get mean values for each metric
df_grp["session"] = pd.Categorical(df_grp["session"], categories=["Saline 1", "Psilocybin", "Saline 2"])
df_mean = df_grp.groupby(["Animal_name", "session"]).mean(numeric_only=True)
df_mean

start         stop    peak_time    peak_power  \
Animal_name session                                                           
Finn        Saline 1    1819.107369  1819.222430  1819.164260  41008.830112   
            Psilocybin  5126.655070  5126.769609  5126.714107  44201.338111   
            Saline 2    1158.382781  1158.488770  1158.437000  31041.087282   
Finn2       Saline 1    5168.725518  5168.844291  5168.784707  28441.120150   
            Psilocybin  7450.831499  7450.941747  7450.886008  30148.335808   
            Saline 2    5039.864368  5039.985288  5039.924510  35405.500568   
Rey         Saline 1    1289.023148  1289.127409  1289.076052  25770.904792   
            Psilocybin  3853.962172  3854.096899  3854.029860  25652.341664   
            Saline 2    2235.835499  2235.937142  2235.888654  44304.921286   
Rose        Saline 1    1442.252948  1442.363153  1442.308559  31125.993390   
            Psilocybin  1465.735327  1465.862987  1465.797633  25155.196785   
            Saline 2    1752.339513  1752.452827  1752.396894  24094.556942   

                        peak_power_abs  duration  peak_frequency_bp  label  \
Animal_name session                                                          
Finn        Saline 1        289.762422  0.115061         168.964438    NaN   
            Psilocybin      288.340668  0.114539         160.939760    NaN   
            Saline 2        218.454593  0.105989         169.660313    NaN   
Finn2       Saline 1        201.782237  0.118773         166.625780    NaN   
            Psilocybin      197.449865  0.110248         161.879168    NaN   
            Saline 2        241.966333  0.120920         168.527166    NaN   
Rey         Saline 1        189.061133  0.104260         170.157315    NaN   
            Psilocybin      173.410084  0.134727         161.701708    NaN   
            Saline 2        319.780045  0.101642         174.030262    NaN   
Rose        Saline 1        230.691436  0.110205         173.027778    NaN   
            Psilocybin      180.437229  0.127660         166.119272    NaN   
            Saline 2        177.963076  0.113315         170.158494    NaN   

                        sharp_wave_amplitude  Animal  
Animal_name session                                   
Finn        Saline 1                6.388298     1.0  
            Psilocybin              5.688975     1.0  
            Saline 2                6.732229     1.0  
Finn2       Saline 1                5.599844     4.0  
            Psilocybin              6.244751     4.0  
            Saline 2                7.490085     4.0  
Rey         Saline 1                8.091481     2.0  
            Psilocybin              8.381922     2.0  
            Saline 2                7.765844     2.0  
Rose        Saline 1                7.854424     3.0  
            Psilocybin              5.397937     3.0  
            Saline 2                6.376375     3.0

In [17]:
# Get ripple counts as well
df_count = df_grp.loc[:, ["session", "Animal_name", "start"]].groupby(["Animal_name", "session"]).count()
df_count.rename(columns={"start": "rpl_count"})

rpl_count
Animal_name session              
Finn        Saline 1          484
            Psilocybin       1739
            Saline 2          372
Finn2       Saline 1         1927
            Psilocybin       3349
            Saline 2         1246
Rey         Saline 1          418
            Psilocybin       1228
            Saline 2         1464
Rose        Saline 1          600
            Psilocybin        656
            Saline 2          673

In [18]:
df_clip = df_grp[(df_grp.start < 3600) & (df_grp.start > 0)]
df_clip

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,Animal_name,Animal
0,0.9184,0.9992,0.9712,28540.330,197.917961,0.0808,168.181818,NaN,13.364659,Saline 1,Finn,1
1,69.9744,70.0368,70.0056,43174.117,319.979066,0.0624,221.212121,NaN,6.180823,Saline 1,Finn,1
2,70.4328,70.5104,70.4736,25572.940,223.388474,0.0776,166.666667,NaN,3.513697,Saline 1,Finn,1
3,89.4656,89.5320,89.5032,35710.438,278.840761,0.0664,171.212121,NaN,6.030679,Saline 1,Finn,1
4,120.9960,121.0720,121.0464,28893.555,223.479148,0.0760,187.878788,NaN,6.569296,Saline 1,Finn,1
...,...,...,...,...,...,...,...,...,...,...,...,...
13380,3538.3128,3538.4280,3538.3464,23109.756,186.316312,0.1152,175.757576,NaN,5.358350,Saline 2,Finn2,4
13381,3554.5184,3554.8744,3554.6696,53843.970,302.665333,0.3560,121.212121,NaN,20.753217,Saline 2,Finn2,4
13382,3569.9776,3570.1144,3570.0640,32305.938,250.608806,0.1368,175.757576,NaN,9.728690,Saline 2,Finn2,4
13383,3570.2416,3570.3416,3570.3096,30171.568,170.430412,0.1000,145.454545,NaN,7.677464,Saline 2,Finn2,4


In [9]:
# Compare to Ilknur data after bugfix
ilknur_new_df = pd.read_csv("/data3/Psilocybin/Recording_Rats/ripple_features_group_from_final_plots.csv", index_col=0)
ilknur_new_df

,start,stop,peak_time,peak_power,peak_power_abs,duration,peak_frequency_bp,label,sharp_wave_amplitude,session,animal,animal_label
0,20.20880,20.31600,20.27200,24817.521,211.327125,0.1072,183.333333,NaN,10.596831,Saline 1,rey,Animal 1
1,33.02320,33.10640,33.07120,18607.932,138.054542,0.0832,153.030303,NaN,36.626322,Saline 1,rey,Animal 1
2,33.14960,33.40560,33.30400,39154.790,281.620276,0.2560,163.636364,NaN,12.076338,Saline 1,rey,Animal 1
3,35.19600,35.24720,35.22000,21863.021,156.924436,0.0512,177.272727,NaN,7.478112,Saline 1,rey,Animal 1
4,44.26720,44.37200,44.34160,25581.988,169.747485,0.1048,115.151515,NaN,6.170822,Saline 1,rey,Animal 1
...,...,...,...,...,...,...,...,...,...,...,...,...
6731,3557.13642,3557.19962,3557.17242,23479.060,169.605681,0.0632,163.636364,NaN,6.141340,Saline 2,finn2,Animal 4
6732,3569.69402,3569.76282,3569.72922,35307.703,279.906008,0.0688,178.787879,NaN,6.294665,Saline 2,finn2,Animal 4
6733,3580.26362,3580.33882,3580.30362,31771.880,213.669109,0.0752,154.545455,NaN,6.274445,Saline 2,finn2,Animal 4
6734,3581.41082,3581.58842,3581.54122,29473.210,234.280787,0.1776,172.727273,NaN,12.184149,Saline 2,finn2,Animal 4


In [ ]:
# Compare to Ilknur data (old)
ilknur_df = pd.read_csv("/Users/nkinsky/Documents/UM/Working/Psilocybin/Recording_Rats/SWR_all_animals_group_data.csv")
ilknur_mean = ilknur_df.groupby(["Animal_name", "session"]).mean(numeric_only=True)
ilknur_mean

In [ ]:
ilknur_m

## Plotting template here with appropriate size graphics and parameters

In [ ]:
%matplotlib widget
features = ["duration", "peak_power", "peak_frequency_bp", "sharp_wave_amplitude"]
titles = ["Duration (s)", "Peak Power", "Peak Frequency", "SW Amplitude"]
fig, axs = plt.subplots(2, 2, figsize=(6, 4))
for idf, feature in enumerate(features):
    ax = axs.reshape(-1)[idf]
    plot_legend = True if idf == 0 else False
    sns.boxplot(data=df_grp, x="Animal", y=feature, dodge=True, hue="session", showfliers=False, fill=False, 
                legend=plot_legend, ax=ax)
    sns.stripplot(data=df_grp, x="Animal", y=feature, dodge=True, hue="session", size=2, 
                  linewidth=0.1, edgecolor="w", alpha=0.3, rasterized=rasterize_scatter,
                  legend=False, ax=ax)
    ax.set_ylabel(titles[idf])
    sns.despine(ax=ax)

fig.savefig(plot_dir / "ripple_features.pdf", dpi=600)

## Other investigations / experimental plots below

### Investigate Rose Psilocybin session - why are peak frequency and power off by so much?!?

In [ ]:
df_rose = df_grp[df_grp.Animal == "Rose"]
df_rose_psi = df_rose[df_rose.session == "Psilocybin"]
_, ax = plt.subplots()
ax.plot(df_rose_psi.peak_time, df_rose_psi.peak_power)

In [ ]:
_, ax = plt.subplots()
sns.lineplot(data=df_mean, x="session", y="duration", hue="Animal", ax=ax)

In [ ]:
%matplotlib widget
features = ["duration", "peak_power", "peak_frequency_bp", "sharp_wave_amplitude"]
_, ax = plt.subplots(2, 2)
for feature, a in zip(features, ax.reshape(-1)):
    # sns.violinplot(data=df_grp, x="session", y=feature, cut=0.1, inner="quart", ax=a)
    sns.boxplot(data=df_grp, x="session", y=feature, showfliers=False, ax=a)
    sns.lineplot(data=df_mean, x="session", y=feature, hue="Animal", ax=a)
    # sal1_df = df_grp[df_grp.session == "Saline 1"]
    # psi_df = df_grp[df_grp.session == "Psilocybin"]
    # sal2_df = df_grp[df_grp.session == "Saline 2"]

    # sal1_v_psi = stats.mannwhitneyu(sal1_df[feature].values, psi_df[feature].values)
    # psi_v_sal2 = stats.mannwhitneyu(psi_df[feature].values, sal2_df[feature].values)
    # sal1_v_sal2 = stats.mannwhitneyu(sal1_df[feature].values, sal2_df[feature].values)

    print(f"FEATURE: {feature}")
    for grps_use in [["Saline 1", "Psilocybin"], ["Psilocybin", "Saline 2"], ["Saline 1", "Saline 2"]]:
        grp1_vals = df_grp[df_grp.session == grps_use[0]][feature].values
        grp2_vals = df_grp[df_grp.session == grps_use[1]][feature].values
        print(f"{grps_use[0]} mean = {np.mean(grp1_vals)}, {grps_use[1]} mean = {np.mean(grp2_vals)}")
        mannu_stats = stats.mannwhitneyu(grp1_vals, grp2_vals)
        print(f"Mann-Whitney U test p-value={mannu_stats.pvalue}\n")
    print("\n")

_, ax = plt.subplots(2, 2)
for feature, a in zip(features, ax.reshape(-1)):
    sns.lineplot(data=df_mean, x="session", y=feature, hue="Animal", ax=a)

In [ ]:
df_split = df_grp[np.isin(df_grp.session.values, ["Saline 1", "Psilocybin"])]

In [ ]:
_, ax = plt.subplots()
df_split = df_grp[np.isin(df_grp.session.values, ["Saline 1", "Psilocybin"])]
sns.violinplot(data=df_split, y="duration", hue="session", split=True, ax=ax)

In [ ]:
%matplotlib widget
features = ["duration", "peak_power", "peak_frequency_bp", "sharp_wave_amplitude"]
_, ax = plt.subplots(2, 2)
for feature, a in zip(features, ax.reshape(-1)):
    sns.boxenplot(data=df_grp, x="session", y=feature, ax=a)